In [1]:
#!npx degit metaplex-foundation/python-api/metaplex metaplex

In [2]:
#hash finder https://magiceden.io/mintlist-tool

#metaboss snapshot mints -c 4P67Vt899BFEFQ4ySk2CtndYcXU1te3QDZkZBWMPFubG

In [3]:
import requests
from metaplex import metadata
from solana.rpc.api import Client
import json
import pandas as pd


In [4]:
import numpy as np

In [5]:
client = Client("http://api.mainnet-beta.solana.com")


In [6]:
def get_atts(mint_id):
    data = metadata.get_metadata(client, mint_id)["data"]
    res = requests.get(data["uri"])
    return res.json()["attributes"]


In [7]:
# read degenape.json
with open("degenape.json") as f:
    degenape = json.load(f)


In [8]:
# get the top 100 nft's from the degenape.json
top_100_nft_traits = [get_atts(nft) for nft in degenape[:10]]


JSONDecodeError: [Errno Expecting value] <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<HTML><HEAD><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">
<TITLE>ERROR: The request could not be satisfied</TITLE>
</HEAD><BODY>
<H1>504 ERROR</H1>
<H2>The request could not be satisfied.</H2>
<HR noshade size="1px">
CloudFront attempted to establish a connection with the origin, but either the attempt failed or the origin closed the connection.
We can't connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.
<BR clear="all">
If you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.
<BR clear="all">
<HR noshade size="1px">
<PRE>
Generated by cloudfront (CloudFront)
Request ID: dHteSwuC2utiN3Qhok4P-zJIk

In [ ]:
# create a dataframe of the top 100 nft's traits
df_all = pd.DataFrame()
for nft in top_100_nft_traits:
    df_1 = pd.DataFrame(nft)
    # add a column for the nft id
    index = top_100_nft_traits.index(nft)
    df_1["nft_id"] = degenape[index]
    # pivot the dataframe on the trait name
    df_1 = df_1.pivot(index="nft_id", columns="trait_type", values="value")

    df_all = df_all.append(df_1)


In [ ]:
df_1 = pd.DataFrame(nft)
# add a column for the nft id
index = top_100_nft_traits.index(nft)
df_1["nft_id"] = degenape[index]
df_1

,trait_type,value,nft_id
0,Background,Purple,2zeBjKaWo8fzPMcgdFBV3mNkzGhdWEeLBb5ucE17kUvv
1,Skin,Red,2zeBjKaWo8fzPMcgdFBV3mNkzGhdWEeLBb5ucE17kUvv
2,EyesWear,Diamond,2zeBjKaWo8fzPMcgdFBV3mNkzGhdWEeLBb5ucE17kUvv
3,Mouth,Skull Bandana,2zeBjKaWo8fzPMcgdFBV3mNkzGhdWEeLBb5ucE17kUvv
4,HeadWear,Plants,2zeBjKaWo8fzPMcgdFBV3mNkzGhdWEeLBb5ucE17kUvv
5,Cloths,Harry Potter,2zeBjKaWo8fzPMcgdFBV3mNkzGhdWEeLBb5ucE17kUvv


In [ ]:
assets_traits = df_all.copy()


In [ ]:
traits = df_all.columns.tolist()


In [ ]:
for trait in traits:
    i_proportion = df_all[trait].value_counts(dropna=False, normalize=True)
    
    assets_traits = assets_traits.merge(
            i_proportion,
            left_on=trait,
            right_index=True,
            suffixes=("", "_rarity"),
            how="left",
        )


In [ ]:
assets_traits.columns

Index(['Background', 'Cloths', 'EyesWear', 'HeadWear', 'Mouth', 'Skin',
       'Background_rarity', 'Cloths_rarity', 'EyesWear_rarity',
       'HeadWear_rarity', 'Mouth_rarity', 'Skin_rarity'],
      dtype='object')

In [ ]:
rarity_list = assets_traits.columns.tolist()
#only include the rarity columns '_rarity'
rarity_list = [col for col in rarity_list if col.endswith("_rarity")]

In [ ]:
rarity = assets_traits[rarity_list].copy()

In [ ]:
factorial_rarity = 1 / rarity[rarity_list]
rarity["factoral_rarity"] = factorial_rarity.sum(axis=1)
rarity = rarity.sort_values("factoral_rarity", ascending=False)
rarity["rarity_rank"] = np.arange(1, rarity.shape[0] + 1, 1)

In [ ]:
rarity

,Background_rarity,Cloths_rarity,EyesWear_rarity,HeadWear_rarity,Mouth_rarity,Skin_rarity,factoral_rarity,rarity_rank
nft_id,,,,,,,,
DuKd3FyLPVjmZJ1esowYCoVj2mgVVDFdWudkX1mgMZJ7,0.1,0.1,0.1,0.1,0.1,0.1,60.000000,1
CxaNXWrY6LMEtMk7b4iTtqUZmLMt8mPLXK6acp14oq5Q,0.2,0.2,0.1,0.1,0.1,0.1,50.000000,2
2zA22nWFZLkKqk3ySH4Hc8aJA8VTDKgPonzby2cjeTYN,0.2,0.2,0.1,0.1,0.1,0.1,50.000000,3
34pPEH6QMY6fxCLtmCXdVVGk6GeZwfgfE3VutqqeYnTZ,0.3,0.1,0.2,0.1,0.1,0.1,48.333333,4
CYsnyjAouTTFy1qHd2iBHVPwuGMPHVboGfJyXFUBaXBr,0.2,0.1,0.1,0.3,0.1,0.1,48.333333,5
9ndqg7rzZoermchb44MxvWcyueCYh18f32Wd2m1dsPHi,0.3,0.1,0.2,0.1,0.2,0.1,43.333333,6
245ZP737DDFiU3FhDwnsHndv7s1vLyHsfDpdSuHsVQg2,0.2,0.2,0.1,0.1,0.2,0.2,40.000000,7
2zeBjKaWo8fzPMcgdFBV3mNkzGhdWEeLBb5ucE17kUvv,0.2,0.1,0.2,0.1,0.3,0.2,38.333333,8
BxK9tyUq96p8Xs9mV3vSg7DZNvFoELrVRmxSuKusaCnk,0.2,0.1,0.1,0.3,0.3,0.2,36.666667,9
